# Expanding your R Skills {#sec-expanding-skills}

Throughout this book, we have covered some popular packages as well as many of the specific functions from these packages. However, it would be impossible to cover all of the packages, functions, and options that are available in R. In this chapter, we will talk about how to use new packages, interpret error messages, debugging, and overall good programming practices to help you take your R programming to the next level.

## Reading Documentation for New Packages

As you start to apply the tools from this book to your own work or in new settings, you may need to install and use new packages or encounter some unexpected errors. Practicing reading package documentation and responding to error messages will help you be able to expand your R skills beyond the topics covered here. We will demonstrate these skills using the **stringr** package [@R-stringr], which is a package that is part of the **tidyverse** and has several functions for dealing with text data.

In [ ]:
#| warning: false

library(tidyverse)
library(HDSinRdata)

Every published package has a CRAN website. This website contains a reference manual that contains the documentation for the functions and data available in the package. Most often, the website also includes useful vignettes that give examples of how to use the functions in the package. The site also tells you what the requirements for using the package are, who the authors of the package are, and when the package was last updated. For example, take a look at the CRAN site for [**stringr**](https://cran.r-project.org/web/packages/stringr/index.html) and read the vignette ["Introduction to String R"](https://cran.r-project.org/web/packages/stringr/vignettes/stringr.html).

We will use this **stringr** package to clean up text related to a PubMed search query for a systematic review. An example search query is given below and is taken from @GUE202131. Our first goal will be to extract the actual search query from the text along with all the terms used in the query. We can assume that the search query will either be fully contained in parentheses or will be a sequence of parenthetical phrases connected with AND or OR. Our goal is to extract the search query as well as all the individual search terms used in the query, but we have to get there in a series of steps.

In [ ]:
sample_str <- " A systematic search will be performed in PubMed, 
Embase, and the Cochrane Library, using the following search query:   
('out-of-hospital cardiac arrest' OR 'OHCA') AND ('MIRACLE 2' OR 
'OHCA' OR 'CAHP' OR 'C-GRAPH' OR 'SOFA' OR 'APACHE' OR 'SAPS’ OR 
’SWAP’ OR ’TTM’)."

The first thing we want to do with the text is clean up the white space by removing any trailing, leading, or repeated spaces. In our example, the string starts with a trailing space and there are also multiple spaces right before the search query. Searching for "white space" in the **stringr** reference manual, we find the `str_trim()` and `str_squish()` functions. Read the documentation for these two functions. You should find that `str_squish()` is the function we are looking for and that it takes a single argument.

In [ ]:
sample_str <- str_squish(sample_str)
sample_str

## Trying Simple Examples

The premise above is a good example of starting with a simple case. Rather than applying your function to your full dataset right away, you want to first make sure that you understand how it works on a simple example on which you can anticipate what the outcome should look like. My next task will be to split the text into words and store this as a character vector. Read the documentation to determine why I used the `str_split_1()` function below. I then double check that the returned result is indeed a vector and print the result.

In [ ]:
sample_str_words <- str_split_1(sample_str, " ")
class(sample_str_words)
sample_str_words

We now want to identify words in this vector that have starting and/or end parentheses. The function `grepl()` takes in a character vector `x` and a pattern to search for. It returns a logical vector for whether or not each element of `x` has a match for that pattern.

In [ ]:
grepl(sample_str_words, ")")

Huh, that didn't match what I expected! I expected to have multiple TRUE/FALSE values outputted - one for each word. Let's read the documentation again.

## Deciphering Error Messages and Warnings

The warning message above will give us a good clue for what went wrong. It says that the inputted pattern has length \> 1. However, the pattern I gave it is a single character. In fact, I specified the arguments in the wrong order. Let's try again. This time I specify `x` and `pattern`.

In [ ]:
grepl(x=sample_str_words, pattern=")")

That worked! However, it won't work if we change that to a starting parentheses. Try it out for yourself to see this. The error message says that it is looking for an end parentheses. In this case, the documentation does not help us. Let's try searching "stringr find start parentheses" using an online search engine. The first search result for me is a [stack overflow question](https://stackoverflow.com/questions/56174805/how-to-search-for-strings-with-parentheses-in-r) that helps us out. We read that we need to use backslashes to tell R to read the parentheses literally rather than as a special character used in a regular expression.

In [ ]:
grepl(x=sample_str_words, pattern="\\(")

When a function doesn't return what we expect it to, it is a good idea to first test whether the arguments we gave it match what we expect, then to re-read the documentation, and then to look for other resources for help. For example, we could check that `sample_str_words` is indeed a character vector, then re-read the **stringr** documentation, and then search our problem.

### Debugging Code

The code below is supposed to extract the search query from the text as well as find the individual search terms used in the query. However, the code is incorrect. You can try out to two test strings given to see why the code output is wrong. Practice reading through the code to understand what it is trying to do. The comments are there to help explain the steps, but you may also want to print the output to figure out what it is doing.

In [ ]:
sample_strA <- " A systematic search will be performed in PubMed, 
Embase, and the Cochrane Library, using the following search query:   
('out-of-hospital cardiac arrest' OR 'OHCA') AND ('MIRACLE 2' OR 
'OHCA' OR 'CAHP' OR 'C-GRAPH' OR 'SOFA' OR 'APACHE' OR 'SAPS’ OR 
’SWAP’ OR ’TTM’)."

sample_strB <- "Searches will be conducted in MEDLINE via PubMed, Web 
of Science, Scopus and Embase. The following search strategy will be 
used:(child OR infant OR preschool child OR preschool children OR 
preschooler OR pre-school child OR pre-school children OR pre school 
child OR pre school children OR pre-schooler OR pre schooler OR 
children OR adolescent OR adolescents)AND(attention deficit disorder 
with hyperactivity OR ADHD OR attention deficit disorder OR ADD OR 
hyperkinetic disorder OR minimal brain disorder) Submitted "

In [ ]:
sample_str <- sample_strB

# separate parentheses, remove extra white space, and split into words
sample_str <- str_replace(sample_str, "\\)", " \\) ")
sample_str <- str_replace(sample_str, "\\(", " \\( ")
sample_str <- str_squish(sample_str)
sample_str_words <- str_split_1(sample_str, " ")

# find indices with parentheses 
end_ps <- grepl(x=sample_str_words, pattern="\\)")
start_ps <- grepl(x=sample_str_words, pattern="\\(")

# find words between first and last parentheses 
search_query <- sample_str_words[ which(end_ps)[1]:which(start_ps)[1]]
search_query <- paste(search_query, collapse=" ")
search_query

# find search terms
search_terms <- str_replace_all(search_query, "\\)", "")
search_terms <- str_replace_all(search_query, "\\(", "")
sample_terms <- str_squish(search_query)
search_terms <- str_split_1(search_terms, " AND | OR ")
search_terms


## General Programming Tips

As you write more complex code and functions, we want to focus on practicing good programming principles. This will help you when you need to share or update your code or when you inevitably run into errors or unexpected behavior. Below are some general programming tips and how they relate to communication and debugging.

1.  **Consistent naming.** Use consistent and informative names for your objects and functions. For example, you can see that within this text we have only used lowercase letters, underscores (`_`), and occasionally numbers in our names. These names should also be informative and unique. This makes it easier to check for typos or duplicate names when debugging. When debugging, check whether you use the same name for different objects or whether you use different names for the same object. You can do this by using the `ls()` function to find all current objects or by checking your environment pane.

In [ ]:
# Recommended
ages <- c(65, 33, 27, 88)
age_mean <- mean(ages)

# Not recommended
x <- c(65, 33, 27, 88)
x1 <- mean(ages)

2.  **Make Your Code Readable.** Readable code requires several elements of communication. Similar to writing an essay, we need to break our code into digestible and structured pieces. First, code should be broken into blocks, using white space to separate steps, and should use correct levels of indentation (one extra level of indentation for each new loop, if/else statement, or function). This means that closing curly braces should be on their own line indicating the end of the block. This will make it easy to check that all parentheses (), brackets \[\], and curly braces {} match. Additionally, you should use line breaks to avoid going over 80 characters on a single line of code. RStudio has an option to reformat or re-indent your code under the Code tab.

    Besides the structure of your code, writing helpful comments and function documentation are key for making your code readable. A good rule of thumb is to write comments for yourself a year from now - you might remember the project goal but you won't remember what `x` represents. You likely do not need a comment for every line of code but you might need one to explain the overall goal of a block of code or to explain any lines of code that do not stand for themselves.

    In the code below, we have not used indentation. This makes it hard to see the structure of the code such as what lines of code are in the loop or if statement. The function does not have any roxygen documentation. However, we have added too many comments. The comments here are repetitive with the code. Last, we named the function `unique()`, which is already a function in R.

In [ ]:
# Not recommended
unique <- function(x){
y <- c() # results
if (length(x) == 0){ # check length 0
return(NULL)} # return NULL
for(i in length(x)){  # loop through x
if(!(x[i] %in% y)){  # check if x[i] in y
y <- c(y, x[i]) }} # add x[i] to y
return(y) # return y
}

    We rewrite the function below addressing these comments. The end result is much easier to read.

In [ ]:
#' Find unique elements of a vector 
#'
#' @param x vector
#' @return new vector with duplicates removed
own_unique <- function(x){
  
  # check for empty vector
  if (length(x) == 0){ 
    return(NULL)
  }
  
  # otherwise y will be unique values
  y <- c()
  
  for(i in 1:length(x)){ 
    # if value of x is not in y, we add it
    if(!(x[i] %in% y)){ 
      y <- c(y, x[i]) 
    }
  }

  return(y) 
}

3.  **Don't Repeat Yourself.** Whenever we are repeating code, we are more likely to introduce errors. Additionally, it makes it hard to update our code later on. When we find ourselves repeating code, we should use a function. If you find yourself repeating constants you should define these values as an object. The code below uses a single line of code to convert categorical variables to factors and stores a vector of which columns are categorical.

In [ ]:
dry_df <- data.frame(age = ages,
                    tb = c(1, 0, 0, 0),
                    heart_rate = c(60, 82, 76, 72),
                    gender = c("Female", "Male", "Nonbinary", 
                               "Female"))

# convert factor variables
cat_vars <- c("tb", "gender")
dry_df[cat_vars] <- lapply(dry_df[cat_vars], factor)

4.  **Practice Reading Documentation.** Whenever we you are using a new function, you should read the documentation first. When debugging, you should check that the input arguments to a function match what is expected and check the examples. Reading these examples will also help when writing your own documentation so you can better understand how to communicate to your audience.

5.  **Start Simple, Build Up.** If we write a large amount of code at once and then it fails to work, it's hard to understand what went wrong. Instead, we should build up our code or functions in small steps and check it after each step. When it comes to testing code, a good mantra is *test early and test often*. So, try to avoid writing too much code before running and checking that the results match what you expect. If you do end up writing a big chunk of code, you can use localize your error by checking the values of objects at different points.

6.  **Get Comfortable Asking for Help.** In software engineering, it's a known tip to have a rubber ducky (or other adorable object) at your desk to talk through your code. Having to verbalize and explain your approach can be really helpful for debugging. R's error messages can sometimes hint at what the error might stem from, but they are not always direct. Searching for error messages you don't understand might give you a better understanding of the problem.

## Exercises

The exercises below focus on reading function documentation and debugging.

1.  Suppose we want to replace the words "Thank you" in the following string with the word "Thanks". Why does the below code fail? How can we correct it?

In [ ]:
string <- "Congratulations on finishing the book! 
Thank you for reading it."
str_sub(string, c(35, 42)) <- "Thanks"
string

2.  The code below uses the `NHANESsample` data from the **HDSinRdata** package. The goal of the code is to plot the worst diastolic blood pressure reading against the worst systolic blood pressure reading for each patient, colored by hypertension status. However, the code currently generates an error message. What is wrong with the code below? There are four errors for you to identify and fix.

In [ ]:
data(NHANESsample)

nhanes_df <- NHANESsample %>% 
  mutate(worst_DBP = max(DBP1, DBP2, DBP3, DBP4), 
     worst_SBP = max(SBP1, SBP2, SBP3, SBP4))

ggplot() %>% 
  geom_point(data = nhanes_df, aes(x = worst_SBP, 
                                   y = worst_DBP), 
             color = HYP)

3.  The code below uses the `breastcancer` data from the **HDSinRdata** package. The goal is to create a logistic regression model for whether or not the diagnosis is benign or malignant and then to create a calibration plot for the model, following the code from [Chapter -@sec-logistic-regression]. What is wrong with the code below? Debug it. Hint: there are three separate errors.

In [ ]:
data(breastcancer)

model <- glm(diagnosis ~ smoothness_worst + symmetry_mean + 
           texture_se + radius_mean, 
         data = breastcancer, family = binomial)

pred_probs <- predict(model)

num_cuts <- 10
calib_data <-  data.frame(prob = pred_probs,
                      bin = cut(pred_probs, breaks = num_cuts),
                      class = mod_start$y)

calib_data <- calib_data %>% 
         group_by(bin) %>% 
         summarize(observed = sum(class)/n(), 
                   expected = sum(prob)/n(), 
                   se = sqrt(observed*(1-observed)/n()))
calib_data

ggplot(calib_data) + 
  geom_abline(intercept = 0, slope = 1, color="red") + 
  geom_errorbar(aes(x = expected, ymin=observed-1.96*se, 
                ymax=observed+1.96*se), 
            colour="black", width=.01)+
  geom_point(aes(x = expected, y = observed)) +
  labs(x="Expected Proportion", y="Observed Proportion") +
  theme_minimal()

---
jupyter: ir
---